# Importing necessary libraries to run models

In [1]:
import numpy as np
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from pickle import load
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


# Need to load my 5 txt files to make the 5 models

In [2]:
def loader(textfile):
    # Open the file as read-only
    file = open(textfile, 'r')
    # Actually read the file
    text = file.read()
    # Now that I have the data in my text variable, need to close out of the file
    file.close()
    return text

In [38]:
# Create the 5 instances of the 5 ratings

one = loader('Four_Star.txt')
# two = loader('Two_Star.txt')
# three = loader('Three_Star.txt')
# four = loader('Four_Star.txt')
# five = loader('Five_Star.txt')

In [39]:
# Need to split up each of these data sets into training sequences by the different lines. 
# End goal is to have user input 5 words and the text generator will work its magic

In [40]:
one_lines = one.split('\n')
# two_lines = two.split('\n')
# three_lines = three.split('\n')
# four_lines = four.split('\n')/
# five_lines = five.split('\n')

In [41]:
# Each line has been broken out with 5 sequence of words
one_lines[:5]

['starsbrand new gastropub with enticing cocktails',
 'new gastropub with enticing cocktails beer',
 'gastropub with enticing cocktails beer and',
 'with enticing cocktails beer and wine',
 'enticing cocktails beer and wine plus']

# Need to convert the words in my data to integers in order to be used by the model. 

Will need to train the keras tokenizer on entire dataset to assign a unique ID to each unique word in the dataset

In [42]:
tokenizer = Tokenizer()

def word_to_int(lines):
    tokenizer.fit_on_texts(lines)
    sequences = tokenizer.texts_to_sequences(one_lines)
    # Need the total size of the vocabulary for our embedding layer in model
    vocab_size = len(tokenizer.word_index) + 1
    return sequences, vocab_size

In [43]:
# Running the function for all 5 data sets
one_sequence, one_vocab_size = word_to_int(one_lines)
# two_sequence, two_vocab_size = word_to_int(two_lines)
# three_sequence, three_vocab_size = word_to_int(three_lines)
# four_sequence, four_vocab_size = word_to_int(four_lines)
# five_sequence, five_vocab_size = word_to_int(five_lines)

In [44]:
# each line has been converted to unique ints to represent each word
len(one_sequence[0])

6

In [45]:
part_one = one_sequence[0:200000]

part_two = one_sequence[200001:400000]

part_three = one_sequence[400001:600000]

part_four = one_sequence[600001:800000]

part_five = one_sequence[800001:1000000]

part_six = one_sequence[1000001:1200000]

part_seven = one_sequence[1200001:-1]

# Need to convert my sequences into inputs and outputs to structure my model to predict words based on previous set of words

## One Star Review Modeling

In [46]:
one_sequences = np.array(one_sequence)


In [47]:
one_sequences

array([[30313,   104,  6174,    12,  3931,   320],
       [  104,  6174,    12,  3931,   320,   236],
       [ 6174,    12,  3931,   320,   236,     2],
       ...,
       [   10,  2213,    23,    21,   365,    40],
       [ 2213,    23,    21,   365,    40,    89],
       [   23,    21,   365,    40,    89,  7304]])

In [48]:
X, y = one_sequences[:,:-1], one_sequences[:,-1]
y = to_categorical(y, num_classes=one_vocab_size)
seq_length = X.shape[1]

In [61]:
# defining my model

model = Sequential()
model.add(Embedding(one_vocab_size, 5, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(one_vocab_size, activation = 'softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 5, 5)              151570    
_________________________________________________________________
lstm_10 (LSTM)               (None, 5, 100)            42400     
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_8 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_9 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_10 (Dense)             (None, 30314)             3061714   
Total params: 3,356,284
Trainable params: 3,356,284
Non-trainable params: 0
_________________________________________________________________


In [62]:
filepath="weights-improvement.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [64]:
model.load_weights('weights-improvement.hdf5')

In [65]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y,batch_size=256, epochs = 97, callbacks=callbacks_list)

Epoch 1/97
 40192/489052 [=>............................] - ETA: 9:30 - loss: 6.3933 - acc: 0.0913

KeyboardInterrupt: 

In [25]:
model.save('model4.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer4.pkl', 'wb'))